# 0. Imports

In [1]:
import utils
import os

# 0. Funções
import re
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# 1. Data Loading
import numpy as np
import pandas as pd

# 2. Gerando DataFrames (teorTexto, orgaoJugador, assuntos, anexos, classeProcesso)
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer

# 3. Tratando a coluna 'anexos' e gerando um Dataframe de anexos
import re
import unicodedata

[nltk_data] Downloading package stopwords to /home/esdras-
[nltk_data]     daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/esdras-
[nltk_data]     daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/esdras-
[nltk_data]     daniel/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/home/esdras-daniel/Documentos/Python/PGM-DataAnalysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-17 10:39:35.146001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739799575.164916  148786 cuda_dnn.cc:831

# 0. Funções

In [2]:
stop_words_pt = set(stopwords.words('portuguese'))

padroes = ['[^a-zA-Z\s]+',
        'poder judiciario do estado do rio grande do norte',
        'poder judiciario estado do rio grande do norte',
        'poder judiciario do estado rio grande do norte',
        'poder judiciario estado rio grande do norte',
        'normal', 'false',
            'ptbr', 'xnone',
            'poder', 'judiciario',
            'estado', 'rio', 'grande', 'norte',
            'vara execucao fiscal',
            'tributaria natal',
            'praca sete setembro',
            'cidade alta',
            'natalrn', 'cep',
            'rn', 'natal', '5902530',
            'assinado', 'digitalmente',
            'forma lei', 'n1141906',
            'comarca', 'forum fazendario',
            'juiz djanirito souza moura',
            'data registrada sistema',
            '1141906', 'veft', '59025275', 'email',
            'secuniefttj', 'jusbr',
            'telefone', 'whatsapp',
            '36738671',
            'judiciario', 'natalpraca',
            'setembro', 'cidade', '59025300',
            'rio norte', 'forum fazendario',
            'juiz', 'djanirito souza mouro',
            'praca alto', 'natalrn', 'nao',
            # Palavras escluidas após a conversa com Nailton dia 07/01/25
            'lei',
    ]

def preprocessamento(text):
    if text == None:
        return text
    
    text_tokens = word_tokenize(text, language='portuguese')

    filtered_text = [word for word in text_tokens if word.lower() not in stop_words_pt and len(word) > 2]
    text = ' '.join(filtered_text)

    # Retira acentos e converte o texto para minúsculo
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower()

    # Retira alguns padrões definidos na lista 'padroes'
    for padrao in padroes:
        text = re.sub(padrao, '', text)

    return text

def limpa_anexos_documentos(text):
    if text == None:
        return text

    # Retira acentos e converte o texto para minúsculo
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower()
    
    text = re.sub(r'[^a-zA-Z;]+', '', text)
    text = text.lower()

    return text

def create_most_common_counter(items_list: np.ndarray, max_features = 10000):
    """Retorna uma lista com os N termos mais frequentes de uma lista de anexos/documentos seprados por ';'"""

    #items_list_flatten = list(itertools.chain(*items_list))

    vectorizer = CountVectorizer(ngram_range=(1,3), max_features=max_features)
    vectorizer.fit_transform(items_list)

    return vectorizer.get_feature_names_out()
    

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_148786/1379930965.py:3: SyntaxWarning: invalid escape sequence '\s'
  padroes = ['[^a-zA-Z\s]+',


# 1. Data Loading

In [3]:
df = pd.read_csv('./data/pgm-dataset-v6.csv', sep=',')
print(f'Shape: {df.shape}')

df.head(3)

Shape: (6400, 12)


,teorTexto,setorDestino,tipoAviso,orgaoJulgador,assuntos,documentos,anexos,classeProcesso,qtd_sentenca,qtd_acordao,qtd_transito_julgado,novoSetorDestino
0,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,APOIO FISCAL,IntimaÆo,5ª Vara de Execução Fiscal e Tributária de Natal,5952,Despacho;Petição;Intimação;Diligência;Penhora;...,0243943-10.2007.8.20.0001 Ext DA;Endereço da e...,1116,0,0,0,APOIO FISCAL
1,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,APOIO FISCAL,IntimaÆo,4ª Vara de Execução Fiscal e Tributária de Natal,10536;5952,Certidão Trânsito em Julgado;Sentença;Petição ...,0865696-23.2018.8.20.5001 Ext DA;Rcda - extrat...,1116,1,0,1,APOIO FISCAL
2,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,APOIO FISCAL,IntimaÆo,4ª Vara de Execução Fiscal e Tributária de Natal,5951,Decisão;Diligência;Mandado;Despacho;Certidão;D...,0508032-92.2006;0508032-92.2006 - EXT;0508032-...,1116,0,0,0,APOIO FISCAL


## 1.1 - Tratando valores nulos

In [4]:
df.isna().sum()

teorTexto               0
setorDestino            0
tipoAviso               0
orgaoJulgador           0
assuntos                3
documentos              0
anexos                  0
classeProcesso          0
qtd_sentenca            0
qtd_acordao             0
qtd_transito_julgado    0
novoSetorDestino        0
dtype: int64

In [5]:
# Substituindo valores nulos das colunas 'assuntos', 'documentos' e 'anexos'

df['assuntos'] = df['assuntos'].fillna(value='Nenhum Assunto')
df['documentos'] = df['documentos'].fillna(value='Nenhum Documento')
df['anexos'] = df['anexos'].fillna(value='Nenhum Anexo')

## 1.2 - Removendo linhas duplicadas

In [6]:
print(f'Contagem de intimações duplicadas por classe\n')
print(df.loc[df.duplicated()]['novoSetorDestino'].value_counts())

Contagem de intimações duplicadas por classe

novoSetorDestino
DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE    28
Procuradoria Fiscal                         21
Outras                                      11
Procuradoria Administrativa                 11
APOIO FISCAL                                 2
Name: count, dtype: int64


## 1.3 - Substituindo valores

In [7]:
df['novoSetorDestino'].value_counts()

novoSetorDestino
APOIO FISCAL                                1280
DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE    1280
Outras                                      1280
Procuradoria Administrativa                 1280
Procuradoria Fiscal                         1280
Name: count, dtype: int64

In [8]:
# Tipo Aviso
swap_dict1 = {'IntimaÆo': 'Intimação',
             'IntimaÆo de Pauta': 'Intimação de Pauta',
             'CitaÆo': 'Citação',
             'NotificaÆo': 'Notificação',
             'IntimaÆo de audincia': 'Intimação de audiência',
             'Intima��o': 'Intimação'}

# Novo Setor Destino
swap_dict2 = {'APOIO FISCAL': 'AF',
              'DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE': 'DCC',
              'Outras': 'Out.',
              'Procuradoria Administrativa': 'PA',
              'Procuradoria Fiscal ': 'PF'}

# Orgão Julgador
swap_dict3 = {
            "Vara de Execução Fiscal e Tributária": "VEFT",
            "Vara da Fazenda Pública": "VFP",
            "Vara da Infância e Juventude": "VIJ",
            "Vara Cível": "VC",
            "Vara de Família e Sucessões": "VF e S",
            "Juizado da Fazenda Pública": "JFP",
            "Juizado Especial da Fazenda Pública": "JEFP",
            "Juizado Especial Cível, Criminal e da Fazenda Pública": "JECFP",
            "Gabinete": "Gab.",
            "Des.": "Des.",
            "Juiz": "J.",
            "Presidente": "Pres.",
            "Câmara Cível": "CC",
            "Turma Recursal": "TR",
            "Comarca de Natal": "Com NAT",
            "Comarca de Mossoró": "Com MOS",
            "Comarca de Parelhas": "Com PAR",
            "Comarca de São Miguel": "Com SMG",
            "Comarca de Apodi": "Com APO",
            "Comarca de Touros": "Com TOU",
            "Comarca de Currais Novos": "Com CN",
            "Pleno": "PL"
        }

df['novoSetorDestino'] = df['novoSetorDestino'].map(swap_dict2, na_action='ignore')
df['tipoAviso'] = df['tipoAviso'].map(swap_dict1, na_action='ignore')
df['orgaoJulgador'] = df['orgaoJulgador'].replace(swap_dict3, regex=True)

In [9]:
# Pré-processamento dos textos
df['teorTexto_clean'] = df['teorTexto'].apply(preprocessamento)



In [15]:
df.drop(['teorTexto', 'anexos', 'setorDestino', 'qtd_sentenca', 'qtd_acordao', 'qtd_transito_julgado'], axis=1, inplace = True)

In [16]:
# Removendo linhas duplicadas
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

# Salvando o csv
if not os.path.exists('./data/pgm-dataset-v6-clean.csv'):
    df.to_csv('./data/pgm-dataset-v6-clean.csv', index=False)

# 2. Gerando DataFrames (teorTexto, orgaoJugador, assuntos, anexos, classeProcesso)

## 2.1 - teorTexto (TF-IDF)

In [12]:
# Aplica TFIDF
vectorizer = TfidfVectorizer(min_df=0.0025, max_df=0.5, max_features=3500)
X_tfidf = vectorizer.fit_transform(df['teorTexto_clean'])

# Cria novo DataFrame
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Salva em CSV
if not os.path.exists('./data/novos_DataFrames/df_tfidf.csv'):
    print('Salvando DataFrame!')
    df_tfidf.to_csv('./data/novos_DataFrames/df_tfidf.csv', index=False)

## 2.2 - orgaoJulgador

In [13]:
# Cria o DataFrame de orgãos julgadores
df_orgaoJulgador = utils.separa_assuntos_documentos(df['orgaoJulgador'])
print(df_orgaoJulgador.shape)

# Salva em CSV
if not os.path.exists('./data/novos_DataFrames/df_orgaoJulgador.csv'):
    print('Salvando DataFrame!')
    df_orgaoJulgador.to_csv('./data/novos_DataFrames/df_orgaoJulgador.csv', index=False)

Número de assuntos/documentos: 62
(6327, 62)


## 2.3 - assuntos

In [14]:
# Cria o DataFrame de assuntos
df_assuntos = utils.separa_assuntos_documentos(df['assuntos'])
print(df_assuntos.shape)

# Salva em CSV
if not os.path.exists('./data/novos_DataFrames/df_assuntos.csv'):
    print('Salvando DataFrame!')
    df_assuntos.to_csv('./data/novos_DataFrames/df_assuntos.csv', index=False)

Número de assuntos/documentos: 379
(6327, 379)


## 2.4 - anexos

In [15]:
# Pré-processamento da coluna anexos
df['anexos_clean'] = df['anexos'].apply(limpa_anexos_documentos)
df.head(3)

,teorTexto,setorDestino,tipoAviso,orgaoJulgador,assuntos,documentos,anexos,classeProcesso,qtd_sentenca,qtd_acordao,qtd_transito_julgado,novoSetorDestino,teorTexto_clean,anexos_clean
0,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,APOIO FISCAL,IntimaÆo,5ª Vara de Execução Fiscal e Tributária de Natal,5952,Despacho;Petição;Intimação;Diligência;Penhora;...,0243943-10.2007.8.20.0001 Ext DA;Endereço da e...,1116,0,0,0,APOIO FISCAL,a contato processo exequente munic...,extda;enderecodaexecutada;informativodoimovel;...
1,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,APOIO FISCAL,IntimaÆo,4ª Vara de Execução Fiscal e Tributária de Natal,10536;5952,Certidão Trânsito em Julgado;Sentença;Petição ...,0865696-23.2018.8.20.5001 Ext DA;Rcda - extrat...,1116,1,0,1,APOIO FISCAL,a processo acao execucao fi...,extda;rcdaextrato;processointimacaoporviaremot...
2,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,APOIO FISCAL,IntimaÆo,4ª Vara de Execução Fiscal e Tributária de Natal,5951,Decisão;Diligência;Mandado;Despacho;Certidão;D...,0508032-92.2006;0508032-92.2006 - EXT;0508032-...,1116,0,0,0,APOIO FISCAL,o direito central avaliacao arrematacao ...,;ext;extdetran


In [31]:
# Encontrando os anexos mais comuns 
anexos_mais_comuns = create_most_common_counter(df['anexos_clean'], max_features=100)
print(len(anexos_mais_comuns))

100


In [32]:
anexos_mais_comuns

array(['anexo', 'anexo anexo', 'anexo anexo anexo', 'ar',
       'ar certidaodedividaativa',
       'ar certidaodedividaativa certidaodedividaativa', 'calculos',
       'cda', 'certidaodedividaativa',
       'certidaodedividaativa certidaodedividaativa',
       'certidaodedividaativa certidaodedividaativa certidaodedividaativa',
       'certidaooab', 'certidaooab cnpj',
       'certidaooab cnpj opcaosimplesnacional', 'cnpj',
       'cnpj opcaosimplesnacional',
       'cnpj opcaosimplesnacional instrucaonormativa',
       'comprovantederesidencia',
       'comprovantederesidencia fichafuncional', 'contestacao',
       'contracheque', 'contrato', 'contratodehonorarios', 'decreto',
       'doc', 'doc doc', 'doc doc doc', 'docacordao',
       'doccomprovantederesidencia',
       'doccomprovantederesidencia docprocuracao',
       'docdocumentospessoais', 'docdocumentospessoais docprocuracao',
       'docescalasdeplantao', 'docfichafinanceira',
       'docfichafinanceiramun', 'docfichafuncio

In [18]:
# Criando uma nova coluna de anexos
data = []

for anexos in df['anexos_clean']:
    new_anexos = ';'.join([anexo for anexo in anexos.split(';') if anexo in anexos_mais_comuns])
    data.append(new_anexos)

df['anexos_clean_filtrados'] = data

In [19]:
# Cria o DataFrame de anexos
df_anexos = utils.separa_assuntos_documentos(df['anexos_clean_filtrados'])
print(df_anexos.shape)

# Salva em CSV
if not os.path.exists('./data/novos_DataFrames/df_anexos.csv'):
    print('Salvando DataFrame!')
    df_anexos.to_csv('./data/novos_DataFrames/df_anexos.csv', index=False)

Número de assuntos/documentos: 3869
(6327, 3869)


In [20]:
df_anexos

,,abaixo,abonodepermanencia,abonomariarosangelaotimizado,abril,abrilcontrachequeemanuelle,ac,acaoanulatoria,acaocivilpublicadeinsulinas,acaocivilpublicajfrnparte,...,volcompressed,volumedaschuvasjaneiro,voto,votodomagistrado,wagnermendoncaebara,whatsappimageat,whatsappvideoat,yjbr,zimbra,zoneamentoambiental
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.5 - classeProcesso

In [21]:
# Cria o DataFrame de classeProcesso
df_classeProcesso = utils.separa_assuntos_documentos(df['classeProcesso'])
print(df_classeProcesso.shape)

# Salva em CSV
if not os.path.exists('./data/novos_DataFrames/df_classeProcesso.csv'):
    print('Salvando DataFrame!')
    df_classeProcesso.to_csv('./data/novos_DataFrames/df_classeProcesso.csv', index=False)

Número de assuntos/documentos: 46
(6327, 46)


In [22]:
df_classeProcesso

,108,111,1114,1116,1118,120,12078,12079,12134,12135,...,460,47,49,63,64,65,66,7,83,90
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6322,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6324,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6325,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 2.6 - Gerando BERT Embeddings de 'teorTexto'

In [26]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(df['teorTexto_clean'])
print(embeddings)

[[ 0.00104966  0.04778648 -0.05706956 ...  0.03200962  0.03886318
  -0.0490778 ]
 [-0.02127992  0.0475069  -0.07261173 ...  0.01911889  0.07137685
   0.016287  ]
 [ 0.00681552 -0.02444817 -0.071715   ...  0.00789872  0.06313203
   0.01621667]
 ...
 [-0.00199818  0.02004013 -0.07222275 ...  0.01150486  0.04351158
   0.01486292]
 [-0.00232636  0.00491925 -0.10001393 ...  0.01055232  0.10587005
   0.04581457]
 [-0.02880604 -0.01112556 -0.09225283 ...  0.05029727  0.06918222
   0.01536387]]


In [30]:
df_embeddings = pd.DataFrame(data=embeddings)

# Salva em CSV
if not os.path.exists('./data/novos_DataFrames/df_bertembeddings.csv'):
    print('Salvando DataFrame!')
    df_classeProcesso.to_csv('./data/novos_DataFrames/df_bertembeddings.csv', index=False)

Salvando DataFrame!


# 2. Criando uma classe que irá fazer o processamento do DataFrame

Principais objetivos da classe:
* Tratar as colunas 'teorTexto', 'orgaoJugador', 'assuntos', 'anexos', 'classeProcesso' e 'novoSetorDestino' para a classificação.

    - **teorTexto**: Será aplicado o TF-IDF para termos as representações dos textos.
    - **orgaoJugador**: <font color='red'>Ainda decidir se irá gerar um Label Encoder ou One-Hot Encoder.</font>
    - **assuntos**: Será feito uma separação usando o caracter ';' (não é necessário pré-processamento nas strings) e retornará uma matriz (n x m) .
        + onde n = número de intimações e m = número de assuntos no DataFrame
    - **anexos**: Será feito uma separação usando o caracter ';' (faz-se nécessário pré-processamento nas strings para reduzir a quantidade de anexos no DataFrame) e retornará uma matriz (n x m).
        + onde n = número de initimações e m = número de anexos no DataFrame
        + Para diminuir o valor de *m* será utilizando regex para remover pontuações, assentos e números. Também será definido um valor máximo para *m*, para chegar nesse valor será montado um vocabulário com os n-gramas mais frequentes, depois será feito um matching em cada anexo para substituir o nome do anexo.
    - **classeProcesso**: <font color='red'>Ainda decidir se irpa gerar um Label Encoder ou One-Hot Encoder.</font>
    - **novoSetorDestino**: Array com os targets para a classificação
    

In [101]:
class Processador_DataFrame():
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.teorTextos_proc = None
        self.orgaoJugador_proc = None
        self.assuntos_proc = None

    def process_teorTexto(self, min_df=0.001, max_df=0.5, max_features=5000):
        try:
            textos = self.df['teorTexto'].to_numpy()
        except KeyError:
            raise KeyError('"teorTexto" não encontrado')
    
        tfidf_vec = TfidfVectorizer(min_df=min_df, max_df=max_df, max_features=max_features)
        temp_matrix = tfidf_vec.fit_transform(textos)

        self.teorTextos_proc = temp_matrix.toarray()

    def process_orgaoJugador(self):
        try:
            orgaos = self.df['orgaoJulgador'].to_numpy()
        except KeyError:
            raise KeyError('"orgaoJugador" não encontrado')
        
        self.orgaoJugador_proc = self.__retorna_matrix_separada_por_char_no_process(orgaos, char=';')

    def process_assuntos(self):
        try:
            assuntos = self.df['assuntos'].to_numpy()
        except KeyError:
            raise KeyError('"assuntos" não encontrado')
        
        self.assuntos_proc = self.__retorna_matrix_separada_por_char_no_process(assuntos)

    def process_anexos(self):
        try:
            anexos = self.df['anexos'].to_numpy()
        except KeyError:
            raise KeyError('"anexos" não encontrado')
        
        

    def __preprocess_column(self, items_list:np.ndarray, char=';', max_features = 10000):
        """Dado uma array de strings separadas pelo caracter ';' retorna as strings normalizadas e substituidas segundo a frequência"""
        
        def limpar_texto(texto):
            texto = re.sub(r'[^a-zA-Z;]+', ' ', texto) # Remove pontuação e números (exceto ;)
            #anexo = re.sub(r'\s+', ' ', anexo).strip()  # Remove espaços extras
            texto = texto.lower() # Converte para minúsculas

            return texto.strip(char)
        
        items_list_clean = [] # Lista que v

        for anexos in items_list:
            items_list_clean.extend(limpar_texto(anexos))
        
    def __retorna_matrix_separada_por_char_no_process(self, items_list:np.ndarray, char=';'):
        """Retorna um dicionário com uma matriz (n x m) separada pelo caracter definido sem pre-processar e os feature_names.
            * Onde:
                n = comprimento da lista
                m = número de elementos individuais encontrados"""
        
        data = []

        # Popula uma lista com as separações
        for item in items_list:
            if isinstance(item, str):
                data.append(item.split(char))
            else:
                data.append([])

        # Obtendo todos os números únicos como colunas
        cols = sorted(set(num for sublist in data for num in sublist))
        print(f'Número de assuntos/documentos: {len(cols)}')

        # Criando a matriz binária
        return {'matriz': np.array([[1 if col in row else 0 for col in cols] for row in data]),
                'feature_names': cols}
